In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import os
%matplotlib inline

np.random.seed(2)

from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')

In [ ]:
#!python -m pip install scikit-learn --user
#!python -m pip install keras
#!python -m pip uninstall tensorflow-gpu==2.0.0-rc1
#!python -m pip show tensorflow

In [ ]:
# Load the data 

DATA_PATH = os.path.join('CharRecog','Data','emnist')
def get_data(filename,path=DATA_PATH):
    file_path = path+filename
    df = pd.read_csv(file_path,header=None)
    data = df.iloc[:,1:]    # Second - end column contain pixel data 
    target = df.iloc[:,0]   # first column is the target 
    del df # Free up Memory
    return target,data

print("Loading Traning Data ...")
Y_train,X_train = get_data('/emnist-bymerge-train.csv')
Y_test,X_test = get_data('/emnist-bymerge-test.csv')
X_train.info()
Y_train.head()

In [ ]:
Y_train.head()

In [ ]:
# Target charter in the dataSet is Stored as int(0-46) Get the int to char Mapping for Merged Data set
# The merged classes, as suggested by the NIST, are for the letters C, I, J, K, L, M, O, P, S, U,V, W, X, Y and Z

def getMapping(file,path = DATA_PATH):
    lbl_dict = {}
    with open(path+file) as f:
        for line in f.readlines():
            line = line.split(' ')
            lbl_dict[int(line[0])] = str(chr(int(line[1])))
    return lbl_dict
lbl_dict = getMapping('/emnist-bymerge-mapping.txt')
print(lbl_dict.items())
def charLabel(x,Lbl_dict):
    return Lbl_dict[x]  

In [ ]:

# Plot the no of data points for different Characters
Y_train.name = "Label"
# covert int to char Label
cY_train = Y_train.apply(charLabel,convert_dtype=False,args=(lbl_dict,)).sort_values()

bg_color = (0.25, 0.25, 0.25)
sns.set(rc={"font.style":"normal",
            "axes.facecolor":bg_color,
            "figure.facecolor":bg_color,
            "text.color":"black",
            "xtick.color":"black",
            "ytick.color":"black",
            "axes.labelcolor":"black",
            "axes.grid":False,
            'axes.labelsize':30,
            'figure.figsize':(20.0, 15.0),
            'xtick.labelsize':25,
            'ytick.labelsize':20})
g = sns.countplot(y = cY_train)


In [ ]:
# check the data for missing values
X_train.isnull().any().describe()

In [ ]:
#X_train.info() # Rows, Columns
X_train.describe()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
def plotImage(n):
    print(cY_train[n]," ->",sep="")
    image = X_train.values[n].reshape([28, 28])
    image = image.transpose()
    sns.set(rc={'figure.figsize':(2.0, 1.0)})
    plt.imshow(image, cmap = mpl.cm.binary)
    plt.show()
plotImage(3)

In [ ]:
def rotate(image):
    image = image.reshape([28, 28])
    image = image = image.transpose()
    return image.reshape([28 * 28])
X_train = np.apply_along_axis(rotate,1,X_train)

In [ ]:
# data vary from [0 - 255] normalize it to [0 - 1]
X_train = X_train / 255.0

In [ ]:
# import numpy as np
# def split_train_test(data, test_ratio):
#     shuffled_indices = np.random.permutation(len(data))
#     test_set_size = int(len(data) * test_ratio)
#     test_indices = shuffled_indices[:test_set_size] train_indices = shuffled_indices[test_set_size:]
#     return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
# reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.reshape(-1,28,28,1)

In [ ]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_train = to_categorical(Y_train, num_classes = 47)

In [ ]:
print(cY_train[2]+ ' -> ')
print(Y_train[2])
g = plt.imshow(X_train[2][:,:,0],origin='upper')


In [ ]:
# drop 1

# X_train = np.delete(X_train, (1), axis=0) # delete row 1
# Y_train = np.delete(Y_train, (1), axis=0)
# cY_train = np.delete(cY_train, (1), axis=0)

# Todo:  Delete operation is resource intencive
# try to change label(Y) to correct value

In [ ]:
Y_train[1] = 

In [ ]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(47, activation = "softmax"))

In [ ]:
# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:

# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
epochs = 1 # Turn epochs to 30 to get high accuracy
batch_size = 86


In [ ]:
# With data augmentation to prevent overfitting 

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)


In [ ]:
# Fit the model
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

In [ ]:

sns.set(rc={'figure.figsize':(20.0, 10.0)})
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_acc'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)


In [4]:
import tensorflow
from tensorflow.python.client import device_lib

SyntaxError: invalid syntax (<ipython-input-4-64cf67ffc8a6>, line 1)

In [3]:
%%time
with tf.device("/device:GPU:0"):
    a = tf.random.uniform([1000,1000])
    b = tf.random.uniform([1000,1000])
c = tf.matmul(a, b)

InternalError: cudaGetDevice() failed. Status: cudaGetErrorString symbol not found.

In [8]:
import joblib
from os import path,listdir
import os, fnmatch
from keras.models import model_from_json


Model_PATH = path.join('CharRecog','Model')

def getModel(filename):
    global Model_PATH
    filename = path.join(Model_PATH,filename)
    print ("Loading Model : ",filename,"\n...")
    if ".joblib" in filename:
        return joblib.load(filename)
    elif ".h5" in filename:
        return loadKerasModel(filename)
        
def TrainModel():
    pass

def loadKerasModel(filename):
    jsonName = filename[:-3]+'.json'
    # load json and create model
    json_file = open(jsonName, 'r')
    model_json = json_file.read()
    myModel = model_from_json(model_json)
    # load weights into new model
    myModel.load_weights(filename)
    return myModel

In [9]:
model = getModel("model_CNN_byMerg.h5")

Loading Model :  CharRecog\Model\model_CNN_byMerg.h5 
...


InternalError: cudaGetDevice() failed. Status: cudaGetErrorString symbol not found.